In [15]:

import os
import io
from pathlib import Path
from typing import Any, Dict, List, Tuple

import h5py
import numpy as np
import pandas as pd
from google.cloud import storage
from google.oauth2 import service_account

# Config (override via .env)
BUCKET_NAME = os.getenv("GCP_BUCKET_NAME", "inkling-ssvep-emg")
BASE_PREFIX = "EMG-nature/data/"
SERVICE_ACCOUNT_KEY = os.getenv(
    "GOOGLE_APPLICATION_CREDENTIALS",
    "./keys/inkling-479911-b140632c44c5.json",
)

print("BUCKET_NAME:", BUCKET_NAME)
print("BASE_PREFIX:", BASE_PREFIX)
print("SERVICE_ACCOUNT_KEY:", SERVICE_ACCOUNT_KEY)


BUCKET_NAME: inkling-ssvep-emg
BASE_PREFIX: EMG-nature/data/
SERVICE_ACCOUNT_KEY: /Users/rayanhasan/code/hildieleyser/Inkling/keys/inkling-479911-fd24b9bdf83e.json


In [16]:

def create_gcs_client(key_path: str = SERVICE_ACCOUNT_KEY) -> storage.Client:
    """
    Create a Google Cloud Storage client using the provided service account key.

    Args:
        key_path (str): Path to a service account JSON key file.

    Returns:
        google.cloud.storage.Client: Authenticated storage client.
    """
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = key_path
    creds = service_account.Credentials.from_service_account_file(key_path)
    return storage.Client(credentials=creds)


In [17]:

def list_participant_folders(client, bucket_name: str = BUCKET_NAME, base_prefix: str = BASE_PREFIX) -> List[str]:
    """
    Return all participant_X/ prefixes under the EMG-nature data root.

    Args:
        client: Google Cloud Storage client.
        bucket_name (str): Bucket to inspect.
        base_prefix (str): Root prefix for EMG-nature data.

    Returns:
        list[str]: Sorted list of participant folder prefixes (with trailing slash).
    """
    prefixes = set()
    for blob in client.list_blobs(bucket_name, prefix=base_prefix):
        parts = blob.name.split("/")
        for i, part in enumerate(parts):
            if part.startswith("participant_"):
                prefixes.add("/".join(parts[: i + 1]) + "/")
    return sorted(prefixes)


In [18]:

def list_block_folders(client, participant_prefix: str, bucket_name: str = BUCKET_NAME) -> List[str]:
    """
    Return all participant_dayX_blockY/ prefixes for a participant.

    Args:
        client: Google Cloud Storage client.
        participant_prefix (str): e.g., "EMG-nature/data/participant_1/".
        bucket_name (str): Bucket to inspect.

    Returns:
        list[str]: Sorted list of block folder prefixes (with trailing slash).
    """
    blocks = set()
    for blob in client.list_blobs(bucket_name, prefix=participant_prefix):
        parts = blob.name.split("/")
        for i, part in enumerate(parts):
            if "block" in part:
                blocks.add("/".join(parts[: i + 1]) + "/")
    return sorted(blocks)


In [19]:

def read_first_dataset_from_bytes(data: bytes):
    """
    Read the first dataset from an in-memory HDF5 byte stream.

    Args:
        data (bytes): Raw HDF5 file contents.

    Returns:
        np.ndarray | None: First dataset in the file, or None if absent.
    """
    with h5py.File(io.BytesIO(data), "r") as h5f:
        keys = list(h5f.keys())
        if not keys:
            return None
        return h5f[keys[0]][()]


In [20]:

def load_block_from_gcs(
    client,
    block_prefix: str,
    participant_id: str,
    bucket_name: str = BUCKET_NAME,
) -> pd.DataFrame:
    """
    Load one block (trials + sensor arrays) directly from GCS into memory.

    Args:
        client: Google Cloud Storage client.
        block_prefix (str): Prefix for the block, e.g., ".../participant_1/day1_block1/".
        participant_id (str): Participant identifier.
        bucket_name (str): Bucket containing the data.

    Returns:
        pandas.DataFrame: One row per trial with raw EMG/glove/finger arrays.
    """
    bucket = client.bucket(bucket_name)
    block_id = block_prefix.rstrip("/").split("/")[-1]

    trials = None
    emg = glove = finger = None

    for blob in bucket.list_blobs(prefix=block_prefix):
        if blob.name.endswith("/"):
            continue
        fname = Path(blob.name).name
        data_bytes = blob.download_as_bytes()

        if fname == "trials.csv":
            trials = pd.read_csv(io.BytesIO(data_bytes))
        elif fname == "emg_data.hdf5":
            emg = read_first_dataset_from_bytes(data_bytes)
        elif fname == "glove_data.hdf5":
            glove = read_first_dataset_from_bytes(data_bytes)
        elif fname == "finger_data.hdf5":
            finger = read_first_dataset_from_bytes(data_bytes)

    lengths = []
    if trials is not None and not trials.empty:
        lengths.append(len(trials))
    for arr in (emg, glove, finger):
        if arr is not None:
            lengths.append(arr.shape[0])

    if not lengths:
        return pd.DataFrame()

    n = min(lengths)

    if trials is None or trials.empty:
        trials = pd.DataFrame({"trial_id": range(n)})

    rows = []
    for i in range(n):
        row = trials.iloc[i].to_dict()
        row.update(
            {
                "participant_id": participant_id,
                "block_id": block_id,
                "emg_raw": emg[i] if emg is not None else None,
                "glove_raw": glove[i] if glove is not None else None,
                "finger_raw": finger[i] if finger is not None else None,
            }
        )
        rows.append(row)

    return pd.DataFrame(rows)


In [21]:

def build_master_dataframe(client) -> pd.DataFrame:
    """
    Build a combined DataFrame for all participants/blocks directly from GCS.

    Args:
        client: Google Cloud Storage client.

    Returns:
        pandas.DataFrame: Concatenated DataFrame of all trials across participants.
    """
    participant_prefixes = list_participant_folders(client)
    all_frames = []

    for pfx in participant_prefixes:
        participant_id = pfx.rstrip("/").split("/")[-1]
        print(f"🔍 Participant: {participant_id}")

        blocks = list_block_folders(client, pfx)
        for block_pfx in blocks:
            print(f"   📦 Block: {block_pfx}")
            df = load_block_from_gcs(client, block_pfx, participant_id)
            if df.empty:
                print("      ⚠️ No data found")
                continue
            all_frames.append(df)
            print(f"      ✅ Rows: {len(df)}")

    if not all_frames:
        return pd.DataFrame()

    return pd.concat(all_frames, ignore_index=True)


In [22]:

client = create_gcs_client()

emg_df = build_master_dataframe(client)

print(emg_df.shape)
print(emg_df.head())


🔍 Participant: participant_1
   📦 Block: EMG-nature/data/participant_1/participant1_day1_block1/
      ✅ Rows: 5
   📦 Block: EMG-nature/data/participant_1/participant1_day1_block2/
      ✅ Rows: 5
   📦 Block: EMG-nature/data/participant_1/participant1_day2_block1/
      ✅ Rows: 5
   📦 Block: EMG-nature/data/participant_1/participant1_day2_block2/
      ✅ Rows: 5
🔍 Participant: participant_2
   📦 Block: EMG-nature/data/participant_2/participant2_day1_block1/
      ✅ Rows: 5
   📦 Block: EMG-nature/data/participant_2/participant2_day1_block2/
      ✅ Rows: 5
   📦 Block: EMG-nature/data/participant_2/participant2_day2_block1/
      ✅ Rows: 5
   📦 Block: EMG-nature/data/participant_2/participant2_day2_block2/
      ✅ Rows: 5
🔍 Participant: participant_3
   📦 Block: EMG-nature/data/participant_3/participant3_day1_block1/
      ✅ Rows: 5
   📦 Block: EMG-nature/data/participant_3/participant3_day1_block2/
      ✅ Rows: 5
   📦 Block: EMG-nature/data/participant_3/participant3_day2_block1/
     

In [23]:
emg_df.head()

,Unnamed: 0,row_number,target_position,grasp,trial_no,block,participant_id,block_id,emg_raw,glove_raw,finger_raw
0,0,0,2,3,0,0,participant_1,participant1_day1_block1,"[3.763498e-05, 1.9842508e-05, 9.071698e-06, 1....","[86.6102294921875, 86.6102294921875, 86.610229...","[16.80420684814453, 16.80420684814453, 16.8042..."
1,1,1,2,3,1,0,participant_1,participant1_day1_block1,"[2.4820081e-05, 2.6200492e-05, 2.5797071e-05, ...","[0.8576509952545166, 0.8576509952545166, 0.857...","[1.467085599899292, 1.467085599899292, 1.46708..."
2,2,2,2,3,2,0,participant_1,participant1_day1_block1,"[7.703583e-06, 9.893e-06, 1.0767478e-05, 9.083...","[2.8023266792297363, 2.8023266792297363, 2.802...","[4.928889751434326, 4.928889751434326, 4.92888..."
3,3,3,2,3,3,0,participant_1,participant1_day1_block1,"[8.394901e-06, 7.721386e-06, 7.572158e-06, 6.6...","[-55.870609283447266, -55.870609283447266, -55...","[6.696066856384277, 6.696066856384277, 6.69606..."
4,4,4,2,3,4,0,participant_1,participant1_day1_block1,"[-1.6433607e-05, -1.6947637e-05, -2.4275832e-0...","[90.43270111083984, 90.43270111083984, 90.4327...","[-0.38769450783729553, -0.38769450783729553, -..."
